<a href="https://colab.research.google.com/github/msmsm104/TIL/blob/main/20220530/%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89_%EC%83%9D%EC%A1%B4%EC%9E%90_%EC%98%88%EC%B8%A1_(Classification)_0530.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source : https://www.kaggle.com/competitions/titanic

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data load

In [ ]:
base_path = '/content/drive/MyDrive/Colab Notebooks/이어드림스쿨/기업강의(fastcampus)/인공지능 머신러닝(0530)/03. Classification/data/'

train = pd.read_csv(base_path + 'train.csv')
test = pd.read_csv(base_path + 'test.csv')
submission = pd.read_csv(base_path + 'gender_submission.csv')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


---

## Data Preprocessing
    1. 결측치 처리
    2. feature selection (분석에 사용하지 않을 column 제거)

In [ ]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
train['Embarked'].value_counts() ## -> 최빈값으로 채운다면 S가 될것이다.

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [ ]:
train[train['Embarked'].isna()]

## 승선한 항구의 이름.

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [ ]:
train[(train['Pclass'] == 1) & (train['Sex'] == 'female')]['Embarked'].value_counts()

## 비슷한 집단이 어떤 항구에서 탓는지 알아보았을때, 역시 S에서 탄 사람이 가장 많으므로 결측치를 S로 채워준다.

S    48
C    43
Q     1
Name: Embarked, dtype: int64

In [ ]:
train.loc[train['Embarked'].isna(), 'Embarked'] = 'S'
# train[(train['Pclass'] == 1) & (train['Sex'] == 'female')]['Embarked'].value_counts()


S    50
C    43
Q     1
Name: Embarked, dtype: int64

In [ ]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [ ]:
## 생존자 예측과 관련 없는 column을 모두 지운다.

train = train.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [ ]:
## Age column의 결측치 처리 (평균값, 최빈값, 추정가능한 데이터로 추정해서 처리...)
## 여기서는 평균값으로 처리

train['Age'] = train['Age'].fillna(train['Age'].mean())

In [ ]:
train.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

---

## Feature Engineering
    1. Categorical feature encoding
    2. Normalization

In [ ]:
train.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C


In [ ]:
# categorical feature --> One-hot Encoding, Ordinal Encoding

train_OHE = pd.get_dummies(train, columns=['Sex', 'Embarked'], drop_first=True) 
## Nominal feature이기 때문에 One-hot Encoding

train_OHE

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,0,3,22.000000,1,0,7.2500,1,0,1
1,1,1,38.000000,1,0,71.2833,0,0,0
2,1,3,26.000000,0,0,7.9250,0,0,1
3,1,1,35.000000,1,0,53.1000,0,0,1
4,0,3,35.000000,0,0,8.0500,1,0,1
...,...,...,...,...,...,...,...,...,...
886,0,2,27.000000,0,0,13.0000,1,0,1
887,1,1,19.000000,0,0,30.0000,0,0,1
888,0,3,29.699118,1,2,23.4500,0,0,1
889,1,1,26.000000,0,0,30.0000,1,0,0


In [ ]:
## Normalization - feature간의 scale을 맞춰주는 방법. (feature간에 단위 차이가 많이 나는 경우.)

X = train_OHE.drop(columns='Survived')  # input matrix
y = train_OHE['Survived']               # target vector

## minmax-scaling (최대 최소) - 0 ~ 1사이의 값으로 scaled 된다.

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)
X_scaled



array([[1.        , 0.27117366, 0.125     , ..., 1.        , 0.        ,
        1.        ],
       [0.        , 0.4722292 , 0.125     , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.32143755, 0.        , ..., 0.        , 0.        ,
        1.        ],
       ...,
       [1.        , 0.36792055, 0.125     , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.32143755, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.39683338, 0.        , ..., 1.        , 1.        ,
        0.        ]])

In [ ]:
X = pd.DataFrame(data=X_scaled, columns=X.columns)

---

## Training

In [ ]:
## classifier model
from sklearn.linear_model import SGDClassifier    # 1. Linear Classifier
from sklearn.linear_model import LogisticRegression    # 2. Logistic Classifier
from sklearn.tree import DecisionTreeClassifier    # 3. Decision Tree (CART)
from sklearn.ensemble import RandomForestClassifier    # 4. Random Forest

## 평가 지표
from sklearn.metrics import accuracy_score


In [ ]:
clf_1 = SGDClassifier()
clf_2 = LogisticRegression()
clf_3 = DecisionTreeClassifier()
clf_4 = RandomForestClassifier()

clf_1.fit(X, y)
clf_2.fit(X, y)
clf_3.fit(X, y)
clf_4.fit(X, y)


## 학습결과의 성능을 관측해보자. (training에 대한 예측결과)
pred_1 = clf_1.predict(X)
pred_2 = clf_2.predict(X)
pred_3 = clf_3.predict(X)
pred_4 = clf_4.predict(X)

In [ ]:
print(f"1. Linear Classifier, Accuracy for traing : %.4f" % accuracy_score(y, pred_1))
print(f"2. Logistic Regression, Accuracy for traing : %.4f" % accuracy_score(y, pred_2))
print(f"3. Decision Tree, Accuracy for traing : %.4f" % accuracy_score(y, pred_3))
print(f"4. Random Forest, Accuracy for traing : %.4f" % accuracy_score(y, pred_4))

1. Linear Classifier, Accuracy for traing : 0.7912
2. Logistic Regression, Accuracy for traing : 0.8002
3. Decision Tree, Accuracy for traing : 0.9820
4. Random Forest, Accuracy for traing : 0.9820


---

## Test (predict)

In [ ]:
## test data 에도 train data에 적용한 것과 동일한 feature engineering을 적용해 줘야한다.
# 1. 결측치 처리
# 2. preprocessing (scaling - minmax)
test = test.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

In [ ]:
test.isna().sum()

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

In [ ]:
train['Age'].mean()

29.699117647058763

In [ ]:
## Train에서 적용한 age 평균값으로 채워야 함!!!<주의>
test['Age'] = test['Age'].fillna(train['Age'].mean())

In [ ]:
test.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        1
Embarked    0
dtype: int64

In [ ]:
test['Fare'] = test['Fare'].fillna(train['Fare'].mean())

In [ ]:
test.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [ ]:
## Encoding 

test_OHE = pd.get_dummies(data=test, columns=['Sex', 'Embarked'], drop_first=True)
test_OHE


,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,3,34.500000,0,0,7.8292,1,1,0
1,3,47.000000,1,0,7.0000,0,0,1
2,2,62.000000,0,0,9.6875,1,1,0
3,3,27.000000,0,0,8.6625,1,0,1
4,3,22.000000,1,1,12.2875,0,0,1
...,...,...,...,...,...,...,...,...
413,3,29.699118,0,0,8.0500,1,0,1
414,1,39.000000,0,0,108.9000,0,0,0
415,3,38.500000,0,0,7.2500,1,0,1
416,3,29.699118,0,0,8.0500,1,0,1


In [ ]:
## Normalize - train의 minmax를 기준으로 (fit 하지 않는다.)

scaled_data = scaler.transform(test_OHE)

In [ ]:
test_OHE = pd.DataFrame(data=scaled_data, columns=test_OHE.columns)

In [ ]:
## 예측(prediction)

result_1 = clf_1.predict(test_OHE)
result_2 = clf_2.predict(test_OHE)
result_3 = clf_3.predict(test_OHE)
result_4 = clf_4.predict(test_OHE)

In [ ]:
## 예측한 결과를 submission에 적용, csv파일을 생성 후 제출하면 완료된다.

submission['Survived'] = result_4
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
submission.to_csv(base_path + 'submission.csv', index=False)